In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn import metrics as m
from thundersvm import SVC as svmgpu
import calculateWeightUsingGa2 as aresult
import pandas as pd
import itertools
import random
from sklearn.ensemble import VotingClassifier
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import VotingClassifier

randomseed = 42
np.random.seed(randomseed)

<IPython.core.display.Javascript object>

1. Read Dataset 
===

In [3]:
data = datasets.load_wine()
x = data.data
y = data.target

print(np.unique(y))

xtrain, xtest, ytrain_original, ytest_original = train_test_split(
    x, y, test_size=0.3, random_state=10
)

ytrain = ytrain_original.copy()
ytest = ytest_original.copy()

[0 1 2]


<IPython.core.display.Javascript object>

In [4]:
original_scores = []
trial1_scores = [] 

from sklearn.model_selection import KFold

kf = KFold(n_splits=10, random_state=randomseed, shuffle=True)
kf.get_n_splits(x)

print(kf)

data_cv = []

for train_index, test_index in kf.split(x):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    data_cv.append([[X_train, X_test], [y_train, y_test]])


for i in range(len(data_cv)):
    print("************************** ==> ", i)
    
    xtrain = data_cv[i][0][0]
    ytrain_original = data_cv[i][1][0]    
    xtest = data_cv[i][0][1]
    ytest_original = data_cv[i][1][1]    
    
    ytrain=ytrain_original.copy()
    ytest=ytest_original.copy() 
    
    # member values
    clf = []
    acc = []
    finalacc = []
    ypredproba_all = []
    ypredconfprob_all = []
    
    # orginal score using random forest classifier
    rf = RandomForestClassifier(random_state=randomseed, n_estimators=10)
    rf.fit(xtrain, ytrain)
    print("original score", m.f1_score(ytest, rf.predict(xtest), average="weighted"))
    original_scores.append(m.f1_score(ytest, rf.predict(xtest), average="weighted"))
    
    
    # generate three base classifers using RF,svm and XGBoost

    #================================================= 

    rf=RandomForestClassifier(random_state=randomseed, n_estimators=10)
    rf.fit(xtrain,ytrain)
    rfpred=rf.predict(xtest)
    print('RF f1_score',m.f1_score(ytest,rfpred,average='weighted'))

    clf.append(rf)
    acc.append(m.f1_score(ytest,rfpred,average='weighted'))
    ypredproba_all.append(rf.predict_proba(xtest))

    confmat=m.confusion_matrix(ytest,rfpred)
    confsumh=np.sum(confmat,axis=1)
    propconfmat=confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[i]= 100*propconfmat[i]/confsumh[i] 
    ypredconfprob_all.append(propconfmat/100)

    #=================================================
    svc=svmgpu(random_state=randomseed,probability=True,C=100,gamma=0.0001)
    svc.fit(xtrain,ytrain)

    svcpred=svc.predict(xtest)
    print('RF svc',m.f1_score(ytest,svcpred,average='weighted'))

    clf.append(svc)
    acc.append(m.f1_score(ytest,svcpred,average='weighted'))
    ypredproba_all.append(svc.predict_proba(xtest))

    confmat=m.confusion_matrix(ytest,svcpred)
    confsumh=np.sum(confmat,axis=1)
    propconfmat=confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[i]= 100*propconfmat[i]/confsumh[i] 
    ypredconfprob_all.append(propconfmat/100)

    #=================================================
    xgbc=xgb.XGBClassifier(random_state=randomseed,n_estimators=100)
    xgbc.fit(xtrain,ytrain)

    xgbpred=xgbc.predict(xtest)
    print('RF xgbc',m.f1_score(ytest,xgbpred,average='weighted'))


    clf.append(xgbc)
    acc.append(m.f1_score(ytest,xgbpred,average='weighted'))
    ypredproba_all.append(xgbc.predict_proba(xtest))

    confmat=m.confusion_matrix(ytest,xgbpred)
    confsumh=np.sum(confmat,axis=1)
    propconfmat=confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[i]= 100*propconfmat[i]/confsumh[i] 
    ypredconfprob_all.append(propconfmat/100)

    # =================================================
    # =================================================
    # generate combinations of features 12,7
    comb = list(itertools.combinations(np.arange(0, 7, 1), 4))

    # generate 10 random numbers
    randnums = []
    for i in range(10):
        randnums.append(random.randrange(0, len(comb)))

    print(randnums)

    comb = np.array(comb)[randnums, :]


    for i in range(len(comb)):
        print(i, " ==================== ", comb[i])

        rf = RandomForestClassifier(random_state=randomseed, n_estimators=50)
        rf.fit(xtrain[:, comb[i]], ytrain)
        rfpred = rf.predict(xtest[:, comb[i]])
        print(m.f1_score(ytest, rfpred, average="weighted"))

        clf.append(rf)
        acc.append(m.f1_score(ytest, rfpred, average="weighted"))
        ypredproba_all.append(rf.predict_proba(xtest[:, comb[i]]))

        confmat = m.confusion_matrix(ytest, rfpred)
        confsumh = np.sum(confmat, axis=0)
        propconfmat = confmat.copy()
        for i in range(propconfmat.shape[0]):
            propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
        ypredconfprob_all.append(propconfmat / 100)

        xgbmodel = xgb.XGBClassifier(random_state=randomseed, n_estimators=50)
        xgbmodel.fit(xtrain, ytrain)
        xgbmodelpred = xgbmodel.predict(xtest)
        print(m.f1_score(ytest, xgbmodelpred, average="weighted"))

        clf.append(xgbmodel)
        acc.append(m.f1_score(ytest, xgbmodelpred, average="weighted"))
        ypredproba_all.append(xgbmodel.predict_proba(xtest))

        confmat = m.confusion_matrix(ytest, xgbmodelpred)
        confsumh = np.sum(confmat, axis=0)
        propconfmat = confmat.copy()
        for i in range(propconfmat.shape[0]):
            propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
        ypredconfprob_all.append(propconfmat / 100)

    # #=================================================
    #Compute the weight using ga and compute the ensemble accuracy
    import calculateWeightUsingGa2 as aresult
    weightvalga=aresult.getbestvalues(acc)

    finalval=0
    for i in range(len(acc)):
        finalval += weightvalga[i]*ypredproba_all[i]

    print('f1_score',m.f1_score(ytest,np.argmax(finalval,axis=1),average='weighted'))
    print('accuracy_score',m.accuracy_score(ytest,np.argmax(finalval,axis=1)))
    trial1_scores.append(m.f1_score(ytest,np.argmax(finalval,axis=1),average='weighted'))




KFold(n_splits=10, random_state=42, shuffle=True)
************************** ==>  0
original score 1.0
RF f1_score 1.0
RF svc 0.9453941120607787
RF xgbc 0.9423280423280423
[34, 21, 33, 9, 1, 13, 17, 5, 3, 20]
0  ====================  [3 4 5 6]
0.7757736091069425
0.9423280423280423
1  ====================  [1 2 3 5]
0.7744539411206077
0.9423280423280423
2  ====================  [2 4 5 6]
0.8867724867724868
0.9423280423280423
3  ====================  [0 1 5 6]
0.8888888888888888
0.9423280423280423
4  ====================  [0 1 2 4]
0.8907407407407406
0.9423280423280423
5  ====================  [0 2 4 5]
1.0
0.9423280423280423
6  ====================  [0 3 4 6]
0.8888888888888888
0.9423280423280423
7  ====================  [0 1 3 5]
0.9453941120607787
0.9423280423280423
8  ====================  [0 1 2 6]
0.8888888888888888
0.9423280423280423
9  ====================  [1 2 3 4]
0.751388888888889
0.9423280423280423
0
1
2
3
4
5
6
7
8
9
14.989298897972308 [0.96913138 0.96157576 0.61016313 0.43

RF svc 0.7764041514041514
RF xgbc 0.9439416792357969
[25, 30, 13, 32, 34, 32, 15, 33, 24, 5]
0  ====================  [1 2 5 6]
0.7716049382716048
0.9439416792357969
1  ====================  [2 3 4 5]
0.7155067155067155
0.9439416792357969
2  ====================  [0 2 4 5]
0.7197083961789844
0.9439416792357969
3  ====================  [2 3 5 6]
0.7716049382716048
0.9439416792357969
4  ====================  [3 4 5 6]
0.7504594820384294
0.9439416792357969
5  ====================  [2 3 5 6]
0.7716049382716048
0.9439416792357969
6  ====================  [0 2 5 6]
0.8858024691358024
0.9439416792357969
7  ====================  [2 4 5 6]
0.7006573673240339
0.9439416792357969
8  ====================  [1 2 4 6]
0.8207471540804874
0.9439416792357969
9  ====================  [0 1 3 5]
0.8318250377073907
0.9439416792357969
0
1
2
3
4
5
6
7
8
9
14.016246684568566 [0.79590722 0.51938461 0.8154823  0.4431039  0.83218438 0.36221602
 0.83876147 0.41358797 0.85004488 0.45606299 0.85566811 0.43557475
 0.8

<IPython.core.display.Javascript object>

In [5]:
print("original socre ", np.mean(original_scores), " std ", np.std(original_scores))
print("ga socre ", np.mean(trial1_scores), " std ", np.std(trial1_scores))

original socre  0.9721687934433032  std  0.03778463610537326
ga socre  0.977431917211329  std  0.03729387609693471


<IPython.core.display.Javascript object>

4. voting classifier
===

In [6]:
ensemb_clf = []
for i in range(len(clf)):
    ensemb_clf.append([str(type(clf[i])) + str(i), clf[i]])

eclf3 = VotingClassifier(estimators=ensemb_clf, voting="soft", flatten_transform=True)

from sklearn.model_selection import cross_val_score
corssvals=cross_val_score(eclf3, x, y, cv=10)


<IPython.core.display.Javascript object>

In [7]:
print("corssvals ", np.mean(corssvals), " std ", np.std(corssvals))

corssvals  0.9666666666666666  std  0.036851386559504457


<IPython.core.display.Javascript object>